In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import random  

slim = tf.contrib.slim

train_dir = "lfw"
my_alpha = 0.2
image_size = 224
BATCH_SIZE = 2

/home/ashutosh/anaconda/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ashutosh/anaconda/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [21]:
sess = tf.Session()
my_triplets = tf.constant(0,tf.float32,shape=[1,3,128])
a = tf.constant(1,tf.float32,shape=[128])
b = tf.constant(2,tf.float32,shape=[128])
c = tf.constant(3,tf.float32,shape=[128])
my_triplet = tf.stack([a, b, c],axis = 0)
# sup = tf.expand_dims(tf.square(tf.subtract(my_triplet[0],my_triplet[1])),axis = 0)
# tf.concat([a, sup], axis = 0)
#my_triplet = tf.concat([tf.expand_dims(my_encodings[index], axis = 0),tf.expand_dims(my_encodings[index+1], axis = 0), tf.expand_dims(my_encodings[t_index], axis = 0)
append_triplet = tf.concat([my_triplets, tf.expand_dims(my_triplet, axis = 0)], axis = 0)
my_triplet_list = tf.unstack(append_triplet)
def triplet_loss(anchor, positive, negative, alpha = 0.2):
    """
    anchor, postive, negative are n-dimensional vectors (tensors)
    alpha - the difference b/w anchor positive and anchor negative pairs; A tensor;
    """
    with tf.name_scope('single_trip_loss'):
        delta_1 = tf.square(tf.subtract(anchor, positive))  #square of euclidean distance or L2 norm
        delta_2 = tf.square(tf.subtract(anchor, negative))
        loss = tf.maximum(tf.reduce_sum(tf.add(tf.subtract(delta_1, delta_2), alpha)), tf.constant(0, dtype = tf.float32))
    return loss

total_loss = 0.0
for my_triplet in my_triplet_list:
    print ("Fuck yeah")
    total_loss += tf.add(triplet_loss(my_triplet[0],my_triplet[1],my_triplet[2]),total_loss)       
total_loss

Fuck yeah
Fuck yeah


<tf.Tensor 'add_1:0' shape=() dtype=float32>

In [2]:
sess = tf.Session()
tf.set_random_seed(1234)
my_list = tf.random_normal([4,128])
# print (sess.run(my_list))
my_list = tf.unstack(my_list)
# print (sess.run(my_list))
# print (sess.run(my_list[0]))

In [3]:
# i0 = tf.constant(0)
# m0 = tf.ones([2, 2])
# c = lambda i, m: i < 10
# b = lambda i, m: [i+1, tf.concat([m, m], axis=0)]
# i1, m1 = tf.while_loop(c, b, loop_vars=[i0, m0],shape_invariants=[i0.get_shape(), tf.TensorShape([None, 2])])
# sess.run(tf.shape(m1))
my_triplets = tf.constant([0],tf.float32, shape = [1,128])
my_triplets = tf.concat([my_triplets, my_triplets],axis = 0)
#sess.run(tf.shape(tf.stack([my_triplets, my_triplets],axis = 0)))
sess.run(tf.shape(tf.concat([my_triplets, my_triplets],axis = 0)))

array([  4, 128], dtype=int32)

In [16]:
def generate_triplets(my_list, sess=None):
    my_triplets = tf.constant(0,tf.float32, shape = [1,128]) #INITIAL TENSOR FOR APPENDING REMOVED LATER

    for index in range(0,(2*BATCH_SIZE),2):
        for t_index in range(2*BATCH_SIZE):
            if((t_index == index) or (t_index == (index+1))):
                continue
            
            else:
                check = tf.less_equal(tf.reduce_sum(tf.square(tf.subtract(my_list[index], my_list[t_index]))),240)
                my_triplets = tf.cond(check, lambda: tf.concat([my_triplets, tf.expand_dims(my_list[index], axis = 0),tf.expand_dims(my_list[index+1], axis = 0), tf.expand_dims(my_list[t_index], axis = 0)], axis = 0) , lambda: my_triplets, name = 'trip_assign_one')
    #my_triplets = tf.stack(my_triplets)  #convert the list into a single tensor
    if sess is not None:
        print('genfunc', sess.run(my_triplets))
    return my_triplets  #Removing the initial [0,:] part of zeros
#my_trips = generate_triplets(my_list)
#print (sess.run([my_trips,tf.shape(my_trips)]))


In [23]:

def triplet_loss(anchor, positive, negative, alpha = 0.2):
    """
    anchor, postive, negative are n-dimensional vectors (tensors)
    alpha - the difference b/w anchor positive and anchor negative pairs; A tensor;
    """
    with tf.name_scope('single_trip_loss'):
        delta_1 = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)))  #square of euclidean distance or L2 norm
        delta_2 = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)))
        loss = tf.maximum(tf.add(tf.subtract(delta_1, delta_2), alpha), tf.constant(0, dtype = tf.float32))
    return loss

def total_triplet_loss(my_list,sess):
    #my_list = tf.unstack(my_list)   #This converts [None,128] tensor into list of [128] shaped tensors and we can iterate over this list now
    total_loss = tf.constant(0,dtype = tf.float32, name = 'initial_trip_loss') 
    my_triplets = generate_triplets(my_list)
    #c = lambda i, loss: i < tf.size(my_triplets) 
    #b = lambda i, loss: [i+3, tf.add(triplet_loss(my_triplets[i],my_triplets[i+1],my_triplets[i+2],loss))] 
    #i1, total_loss = tf.while_loop(c,b,[i0,initial_total_loss])
    i = 1
    while (i<4):
        try:
            total_loss += tf.add(triplet_loss(my_triplets[i],my_triplets[i+1],my_triplets[i+2]),total_loss)
            i = i+3
            print ("alkksnsdlk")
        except:
            break
        
#     for i in range(0, iterator, 2):
#         total_loss += triplet_loss(my_list[(my_triplets[i])],my_list[(my_triplets[i])+1],my_list[(my_triplets[i+1])])
    return total_loss

trip_loss = total_triplet_loss(my_list,sess)
print (sess.run([trip_loss, tf.shape(my_list)]))

alkksnsdlk


InvalidArgumentError: slice index 2 of dimension 0 out of bounds.
	 [[Node: strided_slice_87 = StridedSlice[Index=DT_INT32, T=DT_FLOAT, begin_mask=0, ellipsis_mask=0, end_mask=0, new_axis_mask=0, shrink_axis_mask=1, _device="/job:localhost/replica:0/task:0/device:GPU:0"](trip_assign_one_71/Merge, strided_slice_87/stack, strided_slice_87/stack_1, strided_slice_87/stack_2)]]
	 [[Node: add_25/_217 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_181_add_25", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'strided_slice_87', defined at:
  File "/home/ashutosh/anaconda/envs/tensorflow/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ashutosh/anaconda/envs/tensorflow/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "